# Method of Moments
### Kirill Zakharov
2022

In [1]:
import numpy as np
import pandas as pd
from scipy.optimize import fsolve


In [2]:
data = pd.read_csv('sp500_option.csv')
data = data.iloc[:-1,:]

In [3]:
data = data[['Strike', 'Bid', 'Midpoint', 'Ask']]
data['Strike'] = (data['Strike'].apply(lambda x: x.replace(',', ''))).astype(float)

In [4]:
data['Spread'] = data['Ask'] - data['Bid']

In [5]:
data.head()

,Strike,Bid,Midpoint,Ask,Spread
0,4205.0,309.6,312.05,314.5,4.9
1,4210.0,300.4,305.60,310.8,10.4
2,4215.0,300.2,305.35,310.5,10.3
3,4220.0,290.2,295.75,301.3,11.1
4,4225.0,289.5,296.35,303.2,13.7


In [32]:
q_values = data.Midpoint[1:].values / data.Midpoint[:-1].values

In [33]:
def sample_moment_fun(data, j):
    return 1/len(data) * sum(data**j)

In [34]:
sample_m1 = sample_moment_fun(q_values, 1)
sample_m2 = sample_moment_fun(q_values, 2)
sample_m4 = sample_moment_fun(q_values, 4)
sample_m5 = sample_moment_fun(q_values, 5)

In [67]:
def equations_system(variables):
    r, θ, k, sigma = variables
    
    eq1 = 1 + r - sample_m1
    eq2 = (1 + r)**2 + θ - sample_m2
    eq3 = 1/(k*(k - 2)) * (k**2 * r**4 + 4 * k**2 * r**3 + 6* k**2 * r**2 * θ - 2*k * r**4 + 6*k**2 * r**2 + 12*k**2 * r*θ + 
                           3*k**2 * θ**2 - 8*k* r**3 - 12*k * r**2 * θ + 4* k**2 * r + 6*k**2 * θ - 12*k * r**2 - 24*k*r*θ - 
                           6*k* θ**2 - 3*sigma**2 * θ + k**2 - 8*k*r - 12 *k*θ - 2*k)
    eq4 = 1/(k*(k - 2)) * (k**2 * r**5 + 5*k**2 * r**4 + 10*k**2 * r**3 * θ - 2*k* r**5 + 10*k**2 * r**3 + 30*k**2 * r**2 * θ
                    + 15*k**2 * r * θ**2 - 10* k*r**4 - 20*k*r**3 * θ + 10*k**2 * r**2 + 30* k**2 * r * θ + 15* k**2 * θ**2 - 
                    20* k* r**3 - 60*k*r**2 * θ - 30*k*r*θ**2 - 15*r*sigma**2 * θ + 5*k**2 * r + 10*k**2 * θ - 20*k*r**2 - 60*k*r*θ 
                    - 30*k*θ**2 - 15*sigma**2 * θ + k**2 - 10*k*r - 20*k*θ - 2*k)
    
    return [eq1, eq2, eq3, eq4]
    

In [71]:
parameters_estimation = fsolve(equations_system, (1, 1, 1, 1))

In [72]:
parameters_estimation

array([ 1.16875231e-02, -1.49508211e-01,  1.12202841e+08, -1.80458774e+06])